In [68]:
import numpy as np
import scipy as sp
import math

# Define known parameters
m_dot_f = 0.1840
m_dot_ox = 0.7727
Volume_f = 57.8693e-3
a = 1*2.6676e-5
n = 0.5 
tb = 478.77
rho_fuel = 1522

# Define lower and upper bounds
lb = [44e-3, 200e-3, 0]  # Lower bounds Dpi, Dpf, L
ub = [45e-3, 250e-3, 2.0]  # Upper bounds Dpi, Dpf, L
bounds = [(low, high) for low, high in zip(lb, ub)]
#np.random.seed(100)

# Define the objective function (sum of squared residuals)
def hybrid_equation(x):
    Dpi, Dpf, L = x

    # Fuel Volume Equation
    F1 = Volume_f - (math.pi/4*(Dpf**2-Dpi**2)*L)
    
    # Fuel Mass Flow Rate Equation
    F2 = m_dot_f - (rho_fuel*math.pi*Dpi*L*a*((4*m_dot_ox)/(math.pi*Dpi**2))**n)
    
    # Approximate Burn Time Relation
    F3 = tb - ((Dpf**(2*n+1)-Dpi**(2*n+1))*math.pi**n)/((2*n+1)*a*2**(2*n+1)*m_dot_ox**n)
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F1, F2, F3]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1.0, 1, 1]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = hybrid_equation(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns : Dpi, Dpf, L
x0 = np.array([10e-3, 350e-3, 0.5])

# Basin Hopping Methods: Powell
result = sp.optimize.basinhopping(objective_function,x0=x0,minimizer_kwargs={"method": "Powell", "bounds": bounds},niter=4000,T=0.5,stepsize=0.001,disp=True)

basinhopping step 0: f 1.08323e-10
basinhopping step 1: f 0.000599442 trial_f 0.000599442 accepted 1  lowest_f 1.08323e-10
basinhopping step 2: f 1.08575e-10 trial_f 1.08575e-10 accepted 1  lowest_f 1.08323e-10
basinhopping step 3: f 0.00155572 trial_f 0.00155572 accepted 1  lowest_f 1.08323e-10
basinhopping step 4: f 1.08317e-10 trial_f 1.08317e-10 accepted 1  lowest_f 1.08317e-10
found new global minimum on step 4 with function value 1.08317e-10
basinhopping step 5: f 1.11017e-10 trial_f 1.11017e-10 accepted 1  lowest_f 1.08317e-10
basinhopping step 6: f 1.08316e-10 trial_f 1.08316e-10 accepted 1  lowest_f 1.08316e-10
found new global minimum on step 6 with function value 1.08316e-10
basinhopping step 7: f 1.21686e-10 trial_f 1.21686e-10 accepted 1  lowest_f 1.08316e-10
basinhopping step 8: f 0.00155572 trial_f 0.00155572 accepted 1  lowest_f 1.08316e-10
basinhopping step 9: f 0.00163362 trial_f 0.00163362 accepted 1  lowest_f 1.08316e-10
basinhopping step 10: f 0.000706765 trial_f 0

In [74]:
Dpi, Dpf, L = result.x[0], result.x[1], result.x[2]

print(f"Initial Port Diameter = {Dpi*1000:.4f} mm")
print(f"Final Port Diameter = {Dpf*1000:.4f} mm")
print(f"Fuel Grain Length = {L*1000:.4f} mm")
print(f"Burn time = {((Dpf**(2*n+1)-Dpi**(2*n+1))*math.pi**n)/((2*n+1)*a*2**(2*n+1)*m_dot_ox**n):.4f} s")
print(f"Fuel mass flow rate = {(rho_fuel*math.pi*Dpi*L*a*((4*m_dot_ox)/(math.pi*Dpi**2))**n):.4f} kg/s")
print(f"Fuel grain volume = {(math.pi/4*(Dpf**2-Dpi**2)*L)*1000:.4f} L")
print(f"L/D = {L/Dpf:.4f}")
print(f"Oxidizer mass flux initial= {m_dot_ox/(math.pi/4*Dpi**2):.4f} kg/s-m^3")
L_star = 4
Throat_area = math.pi/4*(44.45*10**-3)**2
Volume_Chamber = L_star*Throat_area
Volume_grain = math.pi/4*(Dpi)**2
Volume_prepost_chamber = (Volume_Chamber-Volume_grain)
Length_prepost_chamber = Volume_prepost_chamber/(math.pi/4*Dpf**2)
#14 multi port
Total_combustor_length = Length_prepost_chamber + L*0.3
print(Volume_Chamber)
print(Volume_grain)
print(Length_prepost_chamber*1000)
print(Total_combustor_length*1000)


Initial Port Diameter = 44.7972 mm
Final Port Diameter = 229.5186 mm
Fuel Grain Length = 1454.3358 mm
Burn time = 478.7700 s
Fuel mass flow rate = 0.1840 kg/s
Fuel grain volume = 57.8792 L
L/D = 6.3365
Oxidizer mass flux initial= 490.2517 kg/s-m^3
0.006207166618944348
0.0015761291712573012
111.93155197355902
548.2322996346875
